In [5]:
def parse_template_data(template_text):
    import re
    import json
    
    headers = re.findall(r'<\|start_header_id\|>(.*?)<\|end_header_id\|>', template_text, re.DOTALL)
    messages = re.findall(r'\<\|start_header_id\|>(.*?)\<\|end_header_id\|>(.*?)<\|eot_id\|>', template_text, re.DOTALL)
    
    conversation_data = {
        'header_ids': {
            'start_header_id': headers[0] if headers else None,
            'end_header_id': headers[1] if len(headers) > 1 else None,
            'eot_id': headers[2] if len(headers) > 2 else None
        },
        'messages': []
    }
    
    for role, content in messages:
        message = {
            'role': role.strip(),
            'content': content.strip()
        }
        conversation_data['messages'].append(message)
    
    return json.dumps(conversation_data, indent=4)

In [6]:
template_text = '''
<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023

{{ if .System }}{{ .System }}
{{- end }}
{{- if .Tools }}When you receive a tool call response, use the output to format an answer to the orginal user question.

You are a helpful assistant with tool calling capabilities.
{{- end }}<|eot_id|>
{{- range $i, $_ := .Messages }}
{{- $last := eq (len (slice $.Messages $i)) 1 }}
{{- if eq .Role "user" }}<|start_header_id|>user<|end_header_id|>
{{- if and $.Tools $last }}

Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{{ range $.Tools }}
{{- . }}
{{ end }}
{{ .Content }}<|eot_id|>
{{- else }}

{{ .Content }}<|eot_id|>
{{- end }}{{ if $last }}<|start_header_id|>assistant<|end_header_id|>

{{ end }}
{{- else if eq .Role "assistant" }}<|start_header_id|>assistant<|end_header_id|>
{{- if .ToolCalls }}
{{ range .ToolCalls }}
{"name": "{{ .Function.Name }}", "parameters": {{ .Function.Arguments }}}{{ end }}
{{- else }}

{{ .Content }}
{{- end }}{{ if not $last }}<|eot_id|>{{ end }}
{{- else if eq .Role "tool" }}<|start_header_id|>ipython<|end_header_id|>

{{ .Content }}<|eot_id|>{{ if $last }}<|start_header_id|>assistant<|end_header_id|>

{{ end }}
{{- end }}
{{- end }}
'''



In [7]:
parsed_data = parse_template_data(template_text)
print(parsed_data)

{
    "header_ids": {
        "start_header_id": "system",
        "end_header_id": "user",
        "eot_id": "assistant"
    },
    "messages": [
        {
            "role": "system",
            "content": "Cutting Knowledge Date: December 2023\n\n{{ if .System }}{{ .System }}\n{{- end }}\n{{- if .Tools }}When you receive a tool call response, use the output to format an answer to the orginal user question.\n\nYou are a helpful assistant with tool calling capabilities.\n{{- end }}"
        },
        {
            "role": "user",
            "content": "{{- if and $.Tools $last }}\n\nGiven the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.\n\nRespond in the format {\"name\": function name, \"parameters\": dictionary of argument name and its value}. Do not use variables.\n\n{{ range $.Tools }}\n{{- . }}\n{{ end }}\n{{ .Content }}"
        },
        {
            "role": "assistant",
            "cont